In [1]:
import tensorflow as tf
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_virtual_device_configuration(
#     physical_devices[0],
#     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1250*3)])
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

# Set random seeds consistently
tf.random.set_seed(7)
np.random.seed(7)

# 0 for benign, 1 for malignant
class_names = ["benign", "malignant"]

def generate_csv(folder, label2int):
    folder_name = Path(folder).name
    labels = list(label2int.keys())
    data_list = []

    for label in labels:
        filepaths = list(Path(folder, label).glob("*"))
        label_int = label2int[label]
        data = [{"filepath": str(filepath), "label": str(label_int)} for filepath in filepaths]
        data_list.extend(data)

    df = pd.DataFrame(data_list)
    output_file = f"{folder_name}.csv"
    df.to_csv(output_file, index=False)
    print(f"Saved {output_file}")

# Generate CSV files
generate_csv("data/train", {"nevus": 0, "seborrheic_keratosis": 0, "melanoma": 1})
generate_csv("data/valid", {"nevus": 0, "seborrheic_keratosis": 0, "melanoma": 1})
generate_csv("data/test", {"nevus": 0, "seborrheic_keratosis": 0, "melanoma": 1})

# Load and preprocess images using data generators
train_metadata_filename = "train.csv"
valid_metadata_filename = "valid.csv"
df_train = pd.read_csv(train_metadata_filename)
df_valid = pd.read_csv(valid_metadata_filename)
n_training_samples = len(df_train)
n_validation_samples = len(df_valid)
print("Number of training samples:", n_training_samples)
print("Number of validation samples:", n_validation_samples)



# Data Augmentation for Training Images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data Augmentation for Validation Images
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

batch_size = 64

# Convert label column values to strings
df_train['label'] = df_train['label'].astype(str)
df_valid['label'] = df_valid['label'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col="filepath",
    y_col="label",
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=7
)

valid_generator = valid_datagen.flow_from_dataframe(
    df_valid,
    x_col="filepath",
    y_col="label",
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

# Build a CNN-based model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=n_training_samples // batch_size,
    validation_data=valid_generator,
    validation_steps=n_validation_samples // batch_size,
    verbose=1,
    epochs=30
)

# Save the model
model.save("skin_condition_classifier_cnn4.h5")

tf.keras.models.save_model(model, 'saved_model_directory6')

Saved train.csv
Saved valid.csv
Saved test.csv


EmptyDataError: No columns to parse from file

In [2]:
tf.keras.models.save_model(model, 'saved_model_directory4')

INFO:tensorflow:Assets written to: saved_model_directory3\assets


INFO:tensorflow:Assets written to: saved_model_directory3\assets


Number of validation samples: 152


Number of testing samples: 600


y_test.shape: (600,)


Evaluating the model...
Loss: 0.49203014373779297   Accuracy: 0.8016666769981384


19/19 [==============================] - 1s 35ms/step
Accuracy after setting the threshold: 0.6283333333333333
